In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import os
import sys
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [3]:
from typing import Any

import dill
import numpy as np

from CrossValidation import cross_validation
from MIRA import CostSensitiveMIRA
from Settings import Settings

from window_based_tagger_config import get_config
from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model_parallel_logged, train_model_parallel, train_model, train_cost_sensitive_instance
from searn_parser_breadth_first import SearnModelBreadthFirst
from causal_model_features import CausalModelType
from feature_extraction import get_features_from_probabilities
from results_procesor import ResultsProcessor

In [4]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
CAUSAL_MODEL_TYPE = CausalModelType.SKIN_CANCER
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

crels_folder = "./crels/SC"
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_TD"

config = get_config(training_folder)
results_processor = ResultsProcessor(dbname="metrics_causal_model_reranker")

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
870 218


In [5]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:12->Result:5',
 'Causer:12->Result:2',
 'Causer:4->Result:11',
 'Causer:5->Result:11',
 'Causer:4->Result:12',
 'Causer:2->Result:2',
 'Causer:6->Result:3',
 'Causer:50->Result:2',
 'Causer:50->Result:3',
 'Causer:2->Result:5']

In [6]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

# Train Re-Ranker

## Extract Parses from Sentence Parser

In [7]:
best_top_n = 2
min_feat_freq = 1
best_max_upd = 2 
best_max_parses = 300
best_min_prob = 0.0  # min prob of 0 seems better

In [8]:
def load_rerank(top_n):
    rr_fname = "xs_rerank_" + str(top_n) + ".dill"
    with open(os.path.join(crels_folder, rr_fname), "rb") as f:
        xs_rerank = dill.load(f)

    rr_fname = "xs_rerank_test" + str(top_n) + ".dill"
    with open(os.path.join(crels_folder, rr_fname), "rb") as f:
        xs_test_rerank = dill.load(f)
    return xs_rerank, xs_test_rerank

## Prepare Features

In [9]:
from filter_features import filter_feats

prefixes = [
    "Prob-",
#     "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_",
    "Propn_",
    "Diff_"
]

# Results for SC
current_best = ['CREL_', 'CChain-', 'Prob-']
best_iterations = 1

# default params (not all of these are optimized)
params = {
    "best_top_n": best_top_n,
    "best_max_upd": best_max_upd,
    "best_max_parses": best_max_parses,
    "best_min_prob": best_min_prob,
    "min_feat_freq": min_feat_freq
}

In [10]:
MONGO_COLLECTION, best_iterations

('SC_RE-RANKER_HYPER_PARAM_TD', 1)

### Load Param Hash (avoid re-running same experiment multiple times)

In [13]:
vals = ("C", "best_max_parses", 
        #"best_max_upd", 
        "max_update_items", 
        "best_min_prob", "best_top_n", "extractors", "initial_weight", "loss_type",\
        "min_feat_freq", "pa_type")

def hash_params(params):
    p = dict()
    # only copy over white list vals
    for v in vals:
        p[v] = params[v]
    return str(sorted(p.items())).replace(" ","")

def load_param_hash(db, collection):
    project = {
        "params": "$parameters",
#         "asof":   "$asof",
        "_id": 1
    }
    feats_pipeline = [{ "$project": project }]
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    print("len(rows)", len(rows))
    param_hash = set()
    for r in rows:
        param_hash.add(hash_params(r["params"]))
    return param_hash


In [14]:
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
param_hash = load_param_hash(client.metrics_causal_model_reranker, MONGO_COLLECTION)
len(param_hash), MONGO_COLLECTION

len(rows) 0


(0, 'SC_RE-RANKER_HYPER_PARAM_TD')

# Hyper Parameter Optimization

In [15]:
import datetime
datetime.datetime.now()

datetime.datetime(2019, 6, 18, 7, 40, 23, 956777)

In [16]:
best_f1 = -1
# best_f1 = 0.7457

In [18]:
# Form a closure to simplify loop
def trn_mdl(top_n, prms, cv_filtrd):
    
    global best_f1, best_C, pa_type, best_max_upd, initial_weight, loss_type
    
    n_jobs = None
    # Uses too much RAM, drop to single threaded
    if top_n > 3:        
        n_jobs = 1
    
    f1 = train_model_parallel_logged(
        training_collection_name=MONGO_COLLECTION, results_processor=results_processor,
        feat_extractors=current_best, params=prms,

        cv_folds=cv_filtrd, name2essay=name2essay, 
        C=best_C, pa_type=pa_type, loss_type=loss_type, max_update_items=best_max_upd, initial_weight=initial_weight,
        set_cr_tags=set_cr_tags,
        # use best iterations from above
        max_epochs=best_iterations, early_stop_iters=best_iterations,
        n_jobs=n_jobs
    )
    if f1 > best_f1:
        best_f1 = f1
        print("New Best F1: {f1:.6f}:\t{params}".format(f1=f1, params=\
                                                       str((top_n, best_C, pa_type, best_max_upd, initial_weight, loss_type))))
    else:
        print("         F1: {f1:.6f}:\t{params}".format(f1=f1, params=\
                                                       str((top_n, best_C, pa_type, best_max_upd, initial_weight, loss_type))))

In [19]:
param_hash = load_param_hash(client.metrics_causal_model_reranker, MONGO_COLLECTION)

for top_n in [1,2,3,5]: # [1,2,3,5] - for SC
    
    print("top_n", top_n)
    xs_rr, _ = load_rerank(top_n)
    xs_temp = get_features_from_probabilities(xs_rr, name2crels, best_max_parses, 
                                     causal_model_type=CAUSAL_MODEL_TYPE,
                                     min_feat_freq=min_feat_freq, min_prob=best_min_prob)
    
    cv_flds_rr = cross_validation(xs_temp, 5)
    cv_flds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_flds_rr]
    
    # Filter feats
    cv_filtered = []
    for tr, test in cv_flds_mm:
        x_tr,x_test = filter_feats(tr, test, current_best)
        cv_filtered.append((x_tr,x_test))
    
    skipped = False
    for best_C in [0.0005, 0.0025, 0.0100, 0.1]:
        for pa_type in [1,2]: # [0]
            for best_max_upd in [1]:
                for initial_weight in [0.01]:
                    for loss_type in ["pb", "ml"]:
                        
                        p = {'C': best_C,
                             'best_max_parses': best_max_parses,
                             'best_min_prob': best_min_prob,
                             'best_top_n': top_n,
                             'extractors': list(current_best),
                             'initial_weight': initial_weight,
                             'loss_type': loss_type,
                             'max_update_items': best_max_upd,
                             'min_feat_freq': min_feat_freq,
                             'pa_type': pa_type
                        }
                        hash_p = hash_params(p)
                        if hash_p in param_hash:
                            print(".", end = '')
                            skipped = True
                            continue
                        if skipped:
                            print()
                        trn_mdl(top_n, p, cv_filtered)
                        skipped = False

# top_n, best_C, pa_type, best_max_upd, initial_weight, loss_type

len(rows) 0
top_n 1
New Best F1: 0.805714:	(1, 0.0005, 1, 1, 0.01, 'pb')
New Best F1: 0.806522:	(1, 0.0005, 1, 1, 0.01, 'ml')
         F1: 0.804152:	(1, 0.0005, 2, 1, 0.01, 'pb')
New Best F1: 0.806736:	(1, 0.0005, 2, 1, 0.01, 'ml')
         F1: 0.799449:	(1, 0.0025, 1, 1, 0.01, 'pb')
         F1: 0.806688:	(1, 0.0025, 1, 1, 0.01, 'ml')
         F1: 0.798843:	(1, 0.0025, 2, 1, 0.01, 'pb')
         F1: 0.806254:	(1, 0.0025, 2, 1, 0.01, 'ml')
         F1: 0.800441:	(1, 0.01, 1, 1, 0.01, 'pb')
         F1: 0.805336:	(1, 0.01, 1, 1, 0.01, 'ml')
         F1: 0.800331:	(1, 0.01, 2, 1, 0.01, 'pb')
         F1: 0.804265:	(1, 0.01, 2, 1, 0.01, 'ml')
         F1: 0.799559:	(1, 0.1, 1, 1, 0.01, 'pb')
         F1: 0.804107:	(1, 0.1, 1, 1, 0.01, 'ml')
         F1: 0.798560:	(1, 0.1, 2, 1, 0.01, 'pb')
         F1: 0.796955:	(1, 0.1, 2, 1, 0.01, 'ml')
top_n 2
         F1: 0.805982:	(2, 0.0005, 1, 1, 0.01, 'pb')
         F1: 0.806574:	(2, 0.0005, 1, 1, 0.01, 'ml')
         F1: 0.802243:	(2, 0.0005, 2, 

## Apply to Test Data

In [ ]:
min_feat_freq, best_min_prob

In [31]:
# f1_score	precision	recall	    best_top_n	C	    best_max_parses	  max_update_items	initial_weight	loss_type	pa_type
# 0.808089	0.856129	0.765153	3	        0.01	300	              1	                0.01	        ml	        1
# # current best settings
# C=best_C, pa_type=pa_type, loss_type=loss_type, max_update_items=best_max_upd, initial_weight=initial_weight,

top_n = 3
best_C = 0.01
best_max_upd = 1
max_update_items = 1
initial_weight = 0.01
loss_type = "ml"
pa_type = 1

In [ ]:
xs_rr, xs_tst_rr = load_rerank(top_n)
xs_temp = get_features_from_probabilities(xs_rr, name2crels, best_max_parses, 
                                 causal_model_type=CAUSAL_MODEL_TYPE,
                                 min_feat_freq=min_feat_freq, min_prob=best_min_prob)

# Don't need CV here, but leave it in anyways
cv_flds_rr = cross_validation(xs_temp, 5)    
xs_test = get_features_from_probabilities(xs_tst_rr, name2crels, best_max_parses, 
                                      causal_model_type=CAUSAL_MODEL_TYPE,
                                      min_feat_freq=min_feat_freq, min_prob=best_min_prob)
xs_train_tmp = []
for train, test in cv_flds_rr:
    xs_train_tmp.extend(test)

xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train_tmp, xs_test)
# Filter at the end
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

## Determine the Optimal Number of Training Iterations

In [33]:
# tuning dataset from training data
num_train = int(0.8 * len(xs_train_mm_fltr))
tmp_train_copy = list(xs_train_mm_fltr)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

In [34]:
%%time
mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=best_max_upd, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=3, train_instance_fn = train_cost_sensitive_instance,
                                                        verbose=True,  early_stopping=True)

Epoch: 0 Train Accuracy: 0.8109 Test Accuracy: 0.8069
Epoch: 1 Train Accuracy: 0.8149 Test Accuracy: 0.8059
Epoch: 2 Train Accuracy: 0.8162 Test Accuracy: 0.8066
Epoch: 3 Train Accuracy: 0.8167 Test Accuracy: 0.8038
Best Test Acc: 0.8069
CPU times: user 1min 26s, sys: 201 ms, total: 1min 26s
Wall time: 1min 26s


In [35]:
best_iterations

1

## Run for X Iterations on the Test Dataset

In [36]:
mdl = CostSensitiveMIRA(C=best_C, pa_type=pa_type, loss_type=loss_type, 
                        max_update_items=best_max_upd, initial_weight=initial_weight)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=best_iterations, early_stop_iters=best_iterations,
    train_instance_fn = train_cost_sensitive_instance, verbose=True, early_stopping=False)

best_iterations

Epoch: 0 Train Accuracy: 0.8123 Test Accuracy: 0.8257
Best Test Acc: 0.8257


1

In [37]:
MONGO_TEST_COLLECTION = "TEST_SC_RE-RANKER_TD"

In [38]:
%%time
test_folds = [(xs_train_mm_fltr, xs_test_mm_fltr)]

params = {
    'C': best_C,
    'best_max_parses': best_max_parses,
    'best_min_prob': best_min_prob,
    'best_top_n': top_n,
    'extractors': list(current_best),
    'initial_weight': initial_weight,
    'loss_type': loss_type,
    'max_update_items': best_max_upd,
    'min_feat_freq': min_feat_freq,
    'pa_type': pa_type
}

test_f1 = train_model_parallel_logged(
        training_collection_name=MONGO_TEST_COLLECTION, results_processor=results_processor,
        feat_extractors=current_best, params=params,

        cv_folds=test_folds, 
        
        name2essay=name2essay, 
        C=best_C, pa_type=pa_type, loss_type=loss_type, max_update_items=best_max_upd, initial_weight=initial_weight,
        set_cr_tags=set_cr_tags,
        # use best iterations from above
        max_epochs=best_iterations, early_stop_iters=best_iterations
    )
print(test_f1)

0.8270422535211268
CPU times: user 28.3 s, sys: 88 ms, total: 28.4 s
Wall time: 28.5 s


In [43]:
len(xs_rr), len(xs_test_mm_fltr)

(870, 218)

In [47]:
# sorted(best_mdl.weights.items(), key = lambda tpl: (tpl[0]))[0:50]

In [49]:
sorted(best_mdl.weights.items(), key = lambda tpl: -abs(tpl[1]))[0:10]

[('Prob-min-prob', 0.22406471653724705),
 ('CREL_Causer:1->Result:2-MAX(prob)', 0.20337475889827955),
 ('Prob-5%-prob', 0.19850412752920427),
 ('Prob-10%-prob', 0.1725022015441227),
 ('CREL_1:50', 0.17),
 ('CREL_Causer:5->Result:50-MAX(prob)', 0.16625748312386457),
 ('CREL_2:50', 0.16),
 ('CREL_1:2', 0.16),
 ('Prob-geo-mean', 0.1558215246665996),
 ('CREL_3:4', 0.15)]